<a href="https://colab.research.google.com/github/jaehyun0220/Colab/blob/master/MediaContensAnalysis_Bert_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#10조. 네이버 댓글 분석을 통한 상위, 하위 클립 예측

### #1. 구글 인증

In [1]:
# Auth 인증 및 Google Drive 활용 Data load
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


### #2. 입력 데이터셋 확인

In [2]:
!ls ../gdrive/My\ Drive/output

file_ep10.csv	 file_ep16.csv	file_ep22.csv  file_ep4.csv
file_ep11.csv	 file_ep17.csv	file_ep23.csv  file_ep5.csv
file_ep12.csv	 file_ep18.csv	file_ep24.csv  file_ep6.csv
file_ep13.csv	 file_ep19.csv	file_ep25.csv  file_ep7.csv
file_ep14_1.csv  file_ep1.csv	file_ep26.csv  file_ep8.csv
file_ep14.csv	 file_ep20.csv	file_ep2.csv   file_ep9.csv
file_ep15.csv	 file_ep21.csv	file_ep3.csv   TheLastEmpress.csv


In [3]:
!pip install regex
# Bert 사용을 위한 설치
!pip install sacremoses sentencepiece 
!pip install pytorch-pretrained-bert
!pip install tools

     |████████████████████████████████| 675kB 6.5MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609184 sha256=4a2e407d8b736ce5611e54563f9b1128a83d8fd796a0d873e78a735fbf3a0d08
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built regex
     |████████████████████████████████| 860kB 6.5MB/s 
     |████████████████████████████████| 1.0MB 39.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=8af43a9501ee4531940d970c5bca26fece60b0166021a97275ae6aac2ce42913
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses
     |████████████████████████████████| 133kB 6.9MB/s 
     |████████████████████████████████| 92kB 5.3MB/s 
  Created wheel for tools: filename=tools-0.1.9-cp36-none-any.whl size=46759 sha256=3b616135f0eb6b7d95261306326ce20cfd55d590f240d0b

In [4]:
# 기본 라이브러리 로드
import pandas as pd
import numpy as np
import os, sys
import regex as re
import random
import io

from tqdm import tqdm_notebook,trange
import tensorflow as tf

#데이터 전처리 관련 라이브러리 로드
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

#파이토치 관련 라이브러리 로드
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# 파이토치 BERT 관련 라이브러리 로드
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification

from tools import *
from multiprocessing import Pool, cpu_count


#모델 평가를 위한 라이브러리 로드
from sklearn import metrics, model_selection
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc

#데이터 분리를 위한 라이브러리 로드
from sklearn.model_selection import train_test_split

#수학 & 통계 관련 라이브러리 로드
import scipy.stats as st
from collections import Counter
import math

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Visualization
import seaborn as sns
import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')

Using TensorFlow backend.


In [5]:
# 입력으로 활용할 데이터셋 List 확인
filelist = os.listdir('../gdrive/My Drive/output')
filelist

['file_ep1.csv',
 'file_ep16.csv',
 'file_ep17.csv',
 'file_ep10.csv',
 'file_ep14.csv',
 'file_ep12.csv',
 'file_ep11.csv',
 'file_ep14_1.csv',
 'file_ep15.csv',
 'file_ep13.csv',
 'file_ep18.csv',
 'file_ep19.csv',
 'file_ep21.csv',
 'file_ep20.csv',
 'file_ep2.csv',
 'file_ep22.csv',
 'file_ep23.csv',
 'file_ep25.csv',
 'file_ep24.csv',
 'file_ep26.csv',
 'file_ep3.csv',
 'file_ep4.csv',
 'file_ep5.csv',
 'file_ep6.csv',
 'file_ep9.csv',
 'file_ep7.csv',
 'TheLastEmpress.csv',
 'file_ep8.csv']

In [6]:
# 총 26회차 491개 하이라이트 클립 존재 (전체 재생수 = 107,221,654 / 클립 당 평균 재생수 = 218,374), 
# 이 중에서 예고편, 미공개, 인터뷰 등 클립 제외하고 총 422회 클립 대상
df_title = pd.read_csv('../gdrive/My Drive/output/TheLastEmpress.csv', encoding = 'euc-kr')
df_title.rename(columns=lambda x: re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》 ]', '', x), inplace=True)

# 조회수 분포를 고려하여 각 회차별로 조회수 상위 4개, 하위 4개 클립을 샘플링 - 총 208개 클립
# 좋아요수, 댓글 수, 댓글 내용, 댓글 작성자 정보 (웹크롤링 통한 추출)

df_ep_tot = pd.DataFrame()

# 상위 하위 클립을 나누어 조회수 내림차순으로 rank를 매김 (1~4는 상위 클립, 5~8은 하위 클립)
for i in tqdm_notebook(filelist):
  if (i[:4] == 'file'):
    df_ep_temp = pd.read_csv('../gdrive/My Drive/output/'+i)
    df_ep_temp['play'] = df_ep_temp['play'].apply(lambda x: int(re.sub(',','', x[4:])))
    df_ep_temp['rank'] = df_ep_temp['play'].rank(method='dense', ascending=False)
    df_ep_tot = df_ep_tot.append(df_ep_temp)

# 크롤링 한 데이터에서 불필요한 열 삭제
df_ep_tot.drop(columns='Unnamed: 0', inplace=True)

# 상위 클립은 1로 하위 클립은 0으로 분류
df_ep_tot['target'] = np.where(df_ep_tot['rank']<=4,1,0)
df_ep_tot

,nick,contents,recomm,unrecomm,title,play,like,reple_count,episode,rank,target
0,rosi****,ㅅㅂ옷입고 목욕탕 들어가는거 ㅈㄴ웃기넼ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ저거 명품일 텐데ㅋㅋㅋㅋ...,5,0,"신은경, 핸드폰 너머 이엘리야의 의도적 신음에 ‘분노 폭발’",474498,842,185,1,1.0,1
1,핑크에메랄드,왠지 선황제가 바람은 못폈을거 같다는 생각이 든다 태후가 황후로 있는한 감히 생각도...,2,0,"신은경, 핸드폰 너머 이엘리야의 의도적 신음에 ‘분노 폭발’",474498,842,185,1,1.0,1
2,서지안,ㅋㅋㅋㅋㅋㅋㅋㅋ,0,0,"신은경, 핸드폰 너머 이엘리야의 의도적 신음에 ‘분노 폭발’",474498,842,185,1,1.0,1
3,경,ㅏ,0,0,"신은경, 핸드폰 너머 이엘리야의 의도적 신음에 ‘분노 폭발’",474498,842,185,1,1.0,1
4,홍홍,이게 나라냐? 방송에서 이딴수위가 나오고 지랄이야 진짜 개좃헬조선 시발,3,5,"신은경, 핸드폰 너머 이엘리야의 의도적 신음에 ‘분노 폭발’",474498,842,185,1,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
1841,Major,와 ㅅㅂ...피지컬봐....,550,1,"이희진, 근육남 최진혁에 공주표 애교 “마이 아포”",155236,645,173,8,8.0,0
1842,스폰지밥,"이쁘십니다 할때 나만설렜냐,,,,,,,",691,4,"이희진, 근육남 최진혁에 공주표 애교 “마이 아포”",155236,645,173,8,8.0,0
1843,박한별,반했네,372,4,"이희진, 근육남 최진혁에 공주표 애교 “마이 아포”",155236,645,173,8,8.0,0
1844,김민정,죄송하지만 이 분 이용합시다 ! 어쨌든 좋은게 좋은거죠 ..,593,4,"이희진, 근육남 최진혁에 공주표 애교 “마이 아포”",155236,645,173,8,8.0,0


### #3. 데이터 정제 - 댓글 중 분석 대상이 되는 contens 내 null data 삭제

In [7]:
df_ep_sample = df_ep_tot.copy()
df_ep_sample.dropna(how='any', inplace=True)
df_ep_sample.info()
df_ep_sample['target'].value_counts() # 3:1 imbalanced dataset (조회수가 많은 클립에 댓글이 많기 때문임))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40801 entries, 0 to 1845
Data columns (total 11 columns):
nick           40801 non-null object
contents       40801 non-null object
recomm         40801 non-null int64
unrecomm       40801 non-null int64
title          40801 non-null object
play           40801 non-null int64
like           40801 non-null object
reple_count    40801 non-null object
episode        40801 non-null int64
rank           40801 non-null float64
target         40801 non-null int64
dtypes: float64(1), int64(5), object(5)
memory usage: 3.7+ MB


1    30632
0    10169
Name: target, dtype: int64

### #4. 데이터 길이 확인

#### #4-1. 데이터 정제 - 정규식 & 형태소 분석

In [34]:
df_ep_sample['contents'] = df_ep_sample['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df_ep_sample.dropna(how='any', inplace=True)
# df_sample_final = df_ep_sample.sample(frac=0.2).copy()
df_sample_final = df_ep_sample[df_ep_sample['episode']==1]
df_sample_final['target'].value_counts()

1    389
0    103
Name: target, dtype: int64

In [0]:
# train_df = df_sample_final[:int(len(df_sample_final)*0.8)]
# test_df = df_sample_final[int(len(df_sample_final)*0.8):]

# display(train_df.tail(2))
# display(test_df.head(2))

# print(train_df['target'].value_counts())
# print(test_df['target'].value_counts())

In [36]:
# Bert 친화 형식으로 변경

all_df_bert = pd.DataFrame({
    'id':range(len(df_sample_final)),
    'label':df_sample_final['target'],
    'alpha':['a']*df_sample_final.shape[0],
    'text': df_sample_final['contents'].replace(r'\n', ' ', regex=True)
})
display(all_df_bert.head())

# train_df_bert = pd.DataFrame({
#     'id':range(len(train_df)),
#     'label':train_df['target'],
#     'alpha':['a']*train_df.shape[0],
#     'text': train_df['contents'].replace(r'\n', ' ', regex=True)
# })

# display(train_df_bert.head())

# dev_df_bert = pd.DataFrame({
#     'id':range(len(test_df)),
#     'label':test_df['target'],
#     'alpha':['a']*test_df.shape[0],
#     'text': test_df['contents'].replace(r'\n', ' ', regex=True)
# })

# display(dev_df_bert.head())

,id,label,alpha,text
0,0,1,a,ㅅㅂ옷입고 목욕탕 들어가는거 ㅈㄴ웃기넼ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ저거 명품일 텐데ㅋㅋㅋㅋ...
1,1,1,a,왠지 선황제가 바람은 못폈을거 같다는 생각이 든다 태후가 황후로 있는한 감히 생각도...
2,2,1,a,ㅋㅋㅋㅋㅋㅋㅋㅋ
3,3,1,a,ㅏ
4,4,1,a,이게 나라냐 방송에서 이딴수위가 나오고 지랄이야 진짜 개좃헬조선 시발


In [37]:
!ls ../gdrive/My\ Drive/data/bert/

all.tsv				cust_model_save		     test.tsv
bert-base-multilingual-cased	dev.tsv			     train.tsv
bert-base-multilingual-uncased	multi_cased_L-12_H-768_A-12  transformers


In [0]:
all_df_bert.to_csv('../gdrive/My Drive/data/bert/all.tsv', sep='\t', index=False, header=False)
# train_df_bert.to_csv('../gdrive/My Drive/data/bert/train.tsv', sep='\t', index=False, header=False)
# dev_df_bert.to_csv('../gdrive/My Drive/data/bert/test.tsv', sep='\t', index=False, header=False)

##  BERT 사용을 위한 사용자 클래스 준비

### Tokenizing 준비

In [0]:
# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = "../gdrive/My Drive/data/bert/"

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-multilingual-cased'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'mca'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'"../gdrive/My Drive/data/bert/cust_model_save/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'"../gdrive/My Drive/data/bert/cust_model_save/{TASK_NAME}_evaluation_report/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = '../gdrive/My Drive/data/bert/multi_cased_L-12_H-768_A-12/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

PRETRAINED_PATH='../gdrive/My Drive/data/bert/bert-base-multilingual-cased/'
# PRETRAINED_PATH='../gdrive/My Drive/data/bert/multi_cased_L-12_H-768_A-12/'

output_mode = OUTPUT_MODE
cache_dir = CACHE_DIR


FILE_VOCAB = "../gdrive/My Drive/data/bert/bert-base-multilingual-cased/bert-base-multilingual-cased-vocab.txt"

In [60]:
!ls ../gdrive/My\ Drive/data/bert/

all.tsv				cust_model_save		     test.tsv
bert-base-multilingual-cased	dev.tsv			     train.tsv
bert-base-multilingual-uncased	multi_cased_L-12_H-768_A-12  transformers


In [62]:
# Create sentence and label lists
df_all = pd.read_csv("../gdrive/My Drive/data/bert/all.tsv", delimiter='\t', header=None, names = ['id', 'label', 'alpha', 'text'])
print(df_all.shape)
df_all.sample(5)
# df_train = pd.read_csv("../gdrive/My Drive/data/bert/train.tsv", delimiter='\t', header=None, names = ['id', 'label', 'alpha', 'text'])
# df_test = pd.read_csv("../gdrive/My Drive/data/bert/test.tsv", delimiter='\t', header=None, names = ['id', 'label', 'alpha', 'text'])
# print(df_train.shape, df_test.shape)
# df_train.sample(5)

(492, 4)


,id,label,alpha,text
78,78,1,a,난 왜 안보이지
55,55,1,a,엄마랑 같이 봤는데 진짜 민망해 죽을 뻔
272,272,1,a,여비서에 대한 인식이 바뀌기는 개뿔 아무리 드라마고 캐릭터상이라지만 시대가 어느땐데...
286,286,1,a,왜항상 드라마는 빙빙저래도노
63,63,1,a,미스함무라비에서 속기사역할 나올때 좋앗는데ㅠ


In [63]:
sentences = df_all.text.values
# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = df_all.label.values
len(sentences)

492

In [0]:
# Load pre-trained model tokenizer (vocabulary)
# tokenizer = BertTokenizer.from_pretrained(FILE_VOCAB, do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained(FILE_VOCAB, do_lower_case=False)
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

## 공통 영역: Word Embedding을 위한 Hyper parameter Tuning

In [65]:
# Hyper Param setting
MAX_LEN = 128

for i in range(0, len(sentences), 2000):
  sentence = sentences[i:(i+2000)]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentence]
  
  if (i % 1000) == 0:
    print(tokenized_texts)
    
  input_ids_tmp = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids_tmp = pad_sequences(input_ids_tmp, maxlen=MAX_LEN, dtype="long", truncating='post', padding='post')
  if i==0:
    input_ids = input_ids_tmp
    print(i)
  else:
    input_ids = np.vstack((input_ids, input_ids_tmp))
    print(i, '완료', input_ids.shape, input_ids_tmp.shape)

[['[CLS]', '[UNK]', '목', '##욕', '##탕', '들어', '##가는', '##거', '[UNK]', '명', '##품', '##일', '[UNK]', '[SEP]'], ['[CLS]', '[UNK]', '선', '##황', '##제가', '바', '##람', '##은', '[UNK]', '같다', '##는', '생', '##각', '##이', '든', '##다', '태', '##후', '##가', '황', '##후', '##로', '있는', '##한', '감', '##히', '생', '##각', '##도', '못', '##했', '##을', '##듯', '[SEP]'], ['[CLS]', '[UNK]', '[SEP]'], ['[CLS]', '[UNK]', '[SEP]'], ['[CLS]', '이', '##게', '나', '##라', '##냐', '방송', '##에서', '이', '##딴', '##수', '##위', '##가', '나', '##오', '##고', '지', '##랄', '##이', '##야', '진', '##짜', '[UNK]', '시', '##발', '[SEP]'], ['[CLS]', '이', '##엘', '##리', '##야', '##님', '진', '##짜', '예', '##쁘', '##다', '[SEP]'], ['[CLS]', '오', '##늘', '##은', '이', '##거', '##다', '[SEP]'], ['[CLS]', '[UNK]', '[UNK]', '[SEP]'], ['[CLS]', '세', '##인', '##데', '저', '##정', '##도는', '[UNK]', '댓', '##글', '##보', '##고', '졸', '##라', '##야', '##한', '##줄', '##알', '##았', '##더', '##니', '##만', '미', '##드', '##세', '##보다', '못', '##하', '##네', '청', '##소', '##년', '##이', '##야', '##기도', '아', '##니',

In [0]:
np.save('wordpiece_tokenize_128.npy', input_ids)

In [67]:
input_ids = np.load('wordpiece_tokenize_128.npy')
print(input_ids.shape)
input_ids[:1]

(492, 128)


array([[   101,    100,   9284, 109971, 119341,  71568,  68828,  41521,
           100,   9281,  52951,  18392,    100,    102,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0, 

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

## Bert

### BERT 사용 관련 Hyperparameter Setting

In [0]:
seed = 42
NUM_LABELS = len(df_sample_final['target'].unique()) #2 If ``config.num_labels > 1`` a classification loss is computed (Cross-Entropy)
training_epochs = 2
lr = 2e-5
batch_size = 16 # Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32

device = torch.device('cuda') # GPU 사용 setting

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [0]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)
train_masks, test_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=seed, test_size=0.1)

# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)
train_masks = torch.tensor(train_masks)
test_masks = torch.tensor(test_masks)

In [0]:
# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [72]:
!ls ../gdrive/My\ Drive/data/bert/bert-base-multilingual-cased

bert-base-multilingual-cased-vocab.txt	pytorch_model.bin
bert_config.json			vocab.txt


In [73]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(PRETRAINED_PATH, num_labels=NUM_LABELS)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
  

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [75]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)

t_total value of -1 results in schedule not being applied


In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in tqdm_notebook(enumerate(train_dataloader)):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in test_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


Train loss: 0.5410628744534084


Epoch:  50%|█████     | 1/2 [00:25<00:25, 25.91s/it]

Validation Accuracy: 0.6875


In [0]:
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

In [0]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [0]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

model.load_state_dict(torch.load(OUTPUT_MODEL_PATH + 'media_bert_model_'+training_epochs+'.bin'))
model.to(device)
for param in model.parameters():
    param.requires_grad=False

bert_eval_net(model, test_loader, device)

In [0]:
def result_summary(pred_y, real_y):
  print("Accuracy: {:.4f}".format(accuracy_score(real_y, pred_y)))
  print("Confusion Matrix: \n", confusion_matrix(real_y, pred_y))
  print("Classification Report Matrix: \n", classification_report(real_y, pred_y, digits=3))

  roc_auc = roc_auc_score(real_y, pred_y)
  print("roc_auc score is : {:.4f}".format(roc_auc))

  fpr, tpr, threshold = roc_curve(real_y, pred_y)
  roc_auc = auc(fpr, tpr)

  auc_graph(roc_auc, fpr, tpr)

def auc_graph(roc_auc, fpr, tpr):
  plt.title('Receiver Operating Characteristic')
  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  plt.legend(loc = 'lower right')
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.ylabel('True Positive Rate')
  plt.xlabel('False Positive Rate')
  plt.show()

In [0]:
import itertools
preds_flat = list(itertools.chain(*preds))
labels_flat = list(itertools.chain(*labels))

result_summary(preds_flat, labels_flat)